# Stock GPT
- Have used to train both forward gpt, backward gpt and daily gpt models.
- Using etf_data_parallel_gpt because it is the most recent codebase.(renamed to motnh-gpt-usethis)
- Training a bigger model because the errors are saturated very easily.
- Running this notebook for ohlcv_data
  

## Step 1 : Import all essentials, make sure they are right

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import torch
from torch import nn
torch.manual_seed(123)
from torch.utils.data import ConcatDataset
# import torch_xla
# import torch_xla.core.xla_model as xm

In [3]:
import os 
import sys
os.environ["CUDA_LAUNCH_BLOCKING"]="1" 

### Model and Data Path

In [4]:
# enter model path
sys.path.append("/kaggle/input/ohlc_daily_gpt/pytorch/default/1")

# enter datapath
sys.path.append("/kaggle/input/all-stock-data")

In [5]:
from ohlcv_tokenizer import OHLCV_Tokenizer, OHLCV_Vocab
from ohlcv_data_cleaner_assembler import OHLCV_DataCleanerAssembler

In [6]:
from stock_tokenizer import (
create_data_loader_for_stock, create_data_loader_for_tokenizer
)

In [7]:
from gpt_config import GPT_CONFIG, set_gpt_config_param
set_gpt_config_param("stock_path",
                    "/kaggle/input/all-stock-data/")
stock_path = GPT_CONFIG["stock_path"]

In [8]:
set_gpt_config_param("save_checkpoint_step", 1000)
set_gpt_config_param("max_vocab", 20_000)
# set_gpt_config_param("rolling_window", 20)
set_gpt_config_param("save_path","/kaggle/working/")
set_gpt_config_param("vocab_size", 20_000 + 1) # hard coded 
set_gpt_config_param("emb_dim", 256)
set_gpt_config_param("n_layers", 6)
set_gpt_config_param("input_batch_size", 256)
set_gpt_config_param("target_batch_size", 1)
set_gpt_config_param("batch_size", 5)
GPT_CONFIG

{'stock_path': '/kaggle/input/all-stock-data/',
 'min_bin': -10000,
 'max_bin': 10000,
 'min_vocab': 0,
 'max_vocab': 20000,
 'bin_size': 50,
 'batch_size': 5,
 'stride': 4,
 'multiply_by': 10000.0,
 'context_length': 256,
 'emb_dim': 256,
 'n_heads': 4,
 'n_layers': 6,
 'dropout_rate': 0.05,
 'qkv_bias': False,
 'predict_new_tokens': 1,
 'train_ratio': 0.9,
 'num_epochs': 1,
 'adamw_learning_rate': 0.0004,
 'adamw_weight_decay': 0.1,
 'save_checkpoint_step': 1000,
 'save_path': '/kaggle/working/',
 'temperature': 1,
 'topk': 7,
 'vocab_size': 20001,
 'input_batch_size': 256,
 'target_batch_size': 1}

In [9]:
# mca = MonthDataCleanerAssembler(
#     cfg = GPT_CONFIG,
#     stock_path=stock_path, 
#     is_batch=True,
#     is_return=False
# )

def func_concat_dataset(folder_name):
    total_concat_dataset = []
    for f in os.listdir(folder_name):
        print(f)
        total_concat_dataset.append( # if monthly, use MonthDataCleanerAssembler
            OHLCV_DataCleanerAssembler(
                cfg = GPT_CONFIG,
                stock_path=os.path.join(stock_path, folder_name, f), # change this 
                is_batch=True,
            ).ConcatDataset()
        )
    return total_concat_dataset
    

### Create Dataloader for training

    1.abc folder 

    2. defgh folder

    3. ijklmn folder
    
    4. opqrst folder 

    5. uvwxyz folder 

In [10]:
all_datasets = func_concat_dataset("/kaggle/input/all-stock-data/abc")
train_loader = create_data_loader_for_tokenizer(
    ConcatDataset(all_datasets),
    cfg = GPT_CONFIG,
    batch_size=GPT_CONFIG["batch_size"],
    shuffle = True,
    num_workers=4 
)

b
a
Unexpected error in file /kaggle/input/all-stock-data/abc/a/amrhw.us.txt, error err=EmptyDataError('No columns to parse from file'), type(err)=<class 'pandas.errors.EmptyDataError'>
Unexpected error in file /kaggle/input/all-stock-data/abc/a/amrh.us.txt, error err=EmptyDataError('No columns to parse from file'), type(err)=<class 'pandas.errors.EmptyDataError'>
Unexpected error in file /kaggle/input/all-stock-data/abc/a/asns.us.txt, error err=EmptyDataError('No columns to parse from file'), type(err)=<class 'pandas.errors.EmptyDataError'>
c


#### Custom Validation Loader for evaluation

In [11]:
_interested_stock = "/kaggle/input/all-stock-data/opqrst/r"

_interested_stock = OHLCV_DataCleanerAssembler(
    cfg = GPT_CONFIG, stock_path=_interested_stock,
    is_batch = True,
)

# debug -- remove after debug 
# train_loader = create_data_loader_for_tokenizer(
#     _interested_stock.ConcatDataset(), 
#     GPT_CONFIG,
#     batch_size=GPT_CONFIG["batch_size"],
#     num_workers=4, 
#     shuffle=True 
# )

custom_val_loader = create_data_loader_for_tokenizer(
    _interested_stock.ConcatDataset(), 
    GPT_CONFIG,
    batch_size=GPT_CONFIG["batch_size"],
    num_workers=4, 
    shuffle=True 
)

Unexpected error in file /kaggle/input/all-stock-data/opqrst/r/rbio.us.txt, error err=EmptyDataError('No columns to parse from file'), type(err)=<class 'pandas.errors.EmptyDataError'>


### GPT Model, Parallelization and device

In [12]:
from ohlcv_gpt_model import OHLCV_GPTModel 
from data_parallel import data_parallel

In [13]:
gpt_model = OHLCV_GPTModel(
    cfg=GPT_CONFIG,
    n_cols = 4
)

# if xm.xla_device().type == "xla":
#     print("using tpu")
#     device = xm.xla_device()
#     gpt_model = gpt_model.to(device)
# else:

gpt_model, device = data_parallel(gpt_model)

print(sum(p.numel() for p in gpt_model.parameters()))

Using CPU
16083200


### Model Training

In [14]:
from stock_train_mechanism import StockTrainer 

trainer = StockTrainer(
    model = gpt_model,
    cfg = GPT_CONFIG,
    load_from_checkpoint=None,
    device = device.type # "cuda" or "cpu"
)

print(trainer)
# see if this uses 2 GPUs or 1 GPU 


            Training Statistics for this exercise >>> 

            Device : cpu
            Total Epochs trained : 0
            Total Steps trained : 0
            Total number of tokens seen so far : 0
            Training Loss : []
            Validation Loss : []
        
        


In [15]:
trainer.engage_training(train_loader=train_loader,
                       num_epochs=1,
                       val_loader=custom_val_loader,
                       eval_freq=100,
                        eval_iter=5
                       )

IndexError: index out of range in self

In [ ]:
trainer.plot_losses()